# MCP Agent

- Requires distributions/masterclass-agents/run.yaml

In [2]:
from llama_stack_client import LlamaStackClient, Agent, AgentEventLogger


In [3]:
client = LlamaStackClient(
    base_url="http://localhost:8321",
)

In [4]:
client.models.list()

INFO:httpx:HTTP Request: GET http://localhost:8321/v1/models "HTTP/1.1 200 OK"


[Model(identifier='llama3.2:3b-instruct-fp16', metadata={}, api_model_type='llm', provider_id='ollama', type='model', provider_resource_id='llama3.2:3b-instruct-fp16', model_type='llm'),
 Model(identifier='all-MiniLM-L6-v2', metadata={'embedding_dimension': 384.0}, api_model_type='embedding', provider_id='ollama', type='model', provider_resource_id='all-minilm:latest', model_type='embedding')]

In [5]:
client.tools.list()

INFO:httpx:HTTP Request: GET http://localhost:8321/v1/tools "HTTP/1.1 200 OK"


[Tool(description='Search the web for information', identifier='web_search', parameters=[Parameter(description='The query to search for', name='query', parameter_type='string', required=True, default=None)], provider_id='tavily-search', tool_host=None, toolgroup_id='builtin::websearch', type='tool', metadata=None, provider_resource_id=None),
 Tool(description='Insert documents into memory', identifier='insert_into_memory', parameters=[], provider_id='rag-runtime', tool_host=None, toolgroup_id='builtin::rag', type='tool', metadata=None, provider_resource_id=None),
 Tool(description='Search for information in a database.', identifier='knowledge_search', parameters=[Parameter(description='The query to search for. Can be a natural language sentence or keywords.', name='query', parameter_type='string', required=True, default=None)], provider_id='rag-runtime', tool_host=None, toolgroup_id='builtin::rag', type='tool', metadata=None, provider_resource_id=None),
 Tool(description='Query Wolfram

In [6]:
client.toolgroups.list()

INFO:httpx:HTTP Request: GET http://localhost:8321/v1/toolgroups "HTTP/1.1 200 OK"


[ToolGroup(identifier='builtin::websearch', provider_id='tavily-search', type='tool_group', args=None, mcp_endpoint=None, provider_resource_id='builtin::websearch'),
 ToolGroup(identifier='builtin::rag', provider_id='rag-runtime', type='tool_group', args=None, mcp_endpoint=None, provider_resource_id='builtin::rag'),
 ToolGroup(identifier='builtin::wolfram_alpha', provider_id='wolfram-alpha', type='tool_group', args=None, mcp_endpoint=None, provider_resource_id='builtin::wolfram_alpha')]

In [12]:
# alternatively can have this in the run.yaml file
client.toolgroups.register(
    toolgroup_id="mcp::filesystem",
    provider_id="model-context-protocol",
    mcp_endpoint={"uri": "http://localhost:8000/sse"},
)

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/toolgroups "HTTP/1.1 200 OK"


In [8]:
client.toolgroups.list()

INFO:httpx:HTTP Request: GET http://localhost:8321/v1/toolgroups "HTTP/1.1 200 OK"


[ToolGroup(identifier='builtin::websearch', provider_id='tavily-search', type='tool_group', args=None, mcp_endpoint=None, provider_resource_id='builtin::websearch'),
 ToolGroup(identifier='builtin::rag', provider_id='rag-runtime', type='tool_group', args=None, mcp_endpoint=None, provider_resource_id='builtin::rag'),
 ToolGroup(identifier='builtin::wolfram_alpha', provider_id='wolfram-alpha', type='tool_group', args=None, mcp_endpoint=None, provider_resource_id='builtin::wolfram_alpha'),
 ToolGroup(identifier='mcp::filesystem', provider_id='model-context-protocol', type='tool_group', args=None, mcp_endpoint=McpEndpoint(uri='http://localhost:8000/sse'), provider_resource_id='mcp::filesystem')]

In [11]:
from llama_stack_client.types.shared_params.agent_config import ToolConfig

agent = Agent(
    client,
    model="llama3.2:3b-instruct-fp16",
    instructions="You are a helpful assistant.",
    tools=["mcp::filesystem"],
)

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/agents "HTTP/1.1 200 OK"


INFO:httpx:HTTP Request: GET http://localhost:8321/v1/tools?toolgroup_id=mcp%3A%3Afilesystem "HTTP/1.1 500 Internal Server Error"
INFO:llama_stack_client._base_client:Retrying request to /v1/tools in 0.475056 seconds
INFO:httpx:HTTP Request: GET http://localhost:8321/v1/tools?toolgroup_id=mcp%3A%3Afilesystem "HTTP/1.1 500 Internal Server Error"
INFO:llama_stack_client._base_client:Retrying request to /v1/tools in 0.750728 seconds
INFO:httpx:HTTP Request: GET http://localhost:8321/v1/tools?toolgroup_id=mcp%3A%3Afilesystem "HTTP/1.1 500 Internal Server Error"


InternalServerError: Error code: 500 - {'detail': 'Internal server error: An unexpected error occurred.'}

In [22]:
user_prompts = [
    "Hello",
    "do you have access to any tools?",
    "list the directories that you have access to",
]

session_id = agent.create_session("test-session")
for prompt in user_prompts:
    print(f"User> {prompt}")
    response = agent.create_turn(
        messages=[{"role": "user", "content": prompt}],
        session_id=session_id,
    )

    for log in AgentEventLogger().log(response):
        log.print()

User> Hello
inference> 

Hello! How can I assist you today?
User> do you have access to any tools?
inference> I do have access to a set of tools that can be invoked to perform various tasks related to file system operations, such as reading and writing files, creating directories, listing directory contents, and more. The list of available tools is provided in the format below:
User> list the directories that you have access to
inference> [list_allowed_directories()]
tool_execution> Tool:list_allowed_directories Args:{}
tool_execution> Tool:list_allowed_directories Response:{"type":"text","text":"Allowed directories:\n/Users/diego/src/ai-masterclass-2025/my-local-files","annotations":null}
inference> The tool `list_allowed_directories()` was invoked successfully. The output indicates that the user has access to a directory named "my-local-files" located at "/Users/diego/src/ai-masterclass-2025".
